In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from classifier import *
from dataloader import *

In [2]:
import multiprocessing as mp

In [3]:
df = get_dataset(data_files,True).drop_duplicates(['TC','TM','D'])
df.head()

,TC,TM,CUT,D,L,ABS,INT,JDK,ICB,DEP,...,TUAPI,UINT,SYNC,CALLSITES,AFPR,RBFA,EXPCAT,CONDCALL,PROJ,IS_MOCK
0,org.apache.hadoop.resourceestimator.translator...,testGetContainerSpec,org.apache.hadoop.resourceestimator.translator...,org.apache.hadoop.yarn.api.records.Resource,real,True,False,False,False,0.5,...,0.0,False,0.0,0.0,0.0,0.0,0.0,0.0,Hadoop,False
1,org.apache.hadoop.resourceestimator.translator...,testGetJobSize,org.apache.hadoop.resourceestimator.translator...,org.apache.hadoop.yarn.api.records.Resource,real,True,False,False,False,0.5,...,0.0,False,0.0,0.0,0.0,0.0,0.0,0.0,Hadoop,False
2,org.apache.hadoop.resourceestimator.translator...,testGetRecurrenceeId,org.apache.hadoop.resourceestimator.translator...,org.apache.hadoop.yarn.api.records.Resource,real,True,False,False,False,0.5,...,0.0,False,0.0,0.0,0.0,0.0,0.0,0.0,Hadoop,False
3,org.apache.hadoop.resourceestimator.translator...,testStringToUnixTimestamp,org.apache.hadoop.resourceestimator.translator...,org.apache.hadoop.yarn.api.records.Resource,real,True,False,False,False,0.5,...,0.0,False,0.0,0.0,0.0,0.0,0.0,0.0,Hadoop,False
4,org.apache.hadoop.resourceestimator.translator...,testResourceSkyline,org.apache.hadoop.resourceestimator.translator...,org.apache.hadoop.yarn.api.records.Resource,real,True,False,False,False,0.5,...,0.0,False,0.0,0.0,0.0,0.0,0.0,0.0,Hadoop,False


In [4]:
df['TC'].count()

543171

In [5]:
pd.DataFrame([
    {
        'PRIOJ': p,
        'TOTAL': df[(df['PROJ']==p)]['PROJ'].count(),
        'MOCK': df[(df['PROJ']==p) & df['IS_MOCK']]['PROJ'].count(),

    }
    for p in projects
])

,PRIOJ,TOTAL,MOCK
0,Hadoop,323078,13484
1,Flink,86141,6719
2,Hive,23368,1490
3,Camel,12936,1855
4,CXF,22550,1489
5,Druid,45332,1869
6,HBase,11966,1072
7,Dubbo,8623,758
8,Oozie,5539,278
9,Storm,3638,369


In [6]:
("%d mocks(%.2f%%)" % (df['IS_MOCK'].sum(), df['IS_MOCK'].sum()/df['IS_MOCK'].count()*100))

'29383 mocks(5.41%)'

In [7]:
training_data = df.copy()

# Training with balanced dataset

In [8]:
def balance_dataset(dataset):
    mocked = dataset[dataset['IS_MOCK']]
    not_mocked = dataset[dataset['IS_MOCK']==False].sample(n=mocked['TC'].count())
    return pd.concat([mocked, not_mocked])

In [9]:
train = training_data
X_train = train.drop(['CUT', 'IS_MOCK', 'TC', 'TM', 'D', 'L', 'PROJ'], axis=1)
y_train = train['IS_MOCK']

In [10]:
classifier = get_model()

In [11]:
from sklearn2pmml.pipeline import PMMLPipeline
from sklearn2pmml import sklearn2pmml

pipeline = PMMLPipeline([('GB', classifier)])
pipeline.fit(X_train, y_train)

sklearn2pmml(pipeline, 'mock-sniffer.pmml')